In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [4]:
cd 

/Users/dorislee/Google Drive


In [9]:
# csv files for answer responses
gdrive_path = "/Users/dorislee/Google Drive/Turn/VAST/user_study/evaluation_study/transcription_and_analysis/"
# user study logs
box_path = "/Users/dorislee/Box/viz-summarization-user-study-material/" 

In [6]:
user_task =  pd.read_csv(path+"UserTaskAssignment.csv")

In [10]:
box_path

'/Users/dorislee/Box/viz-summarization-user-study-material/'